In [2]:
import pandas as pd
import psycopg2
import psycopg2.extras
from psycopg2.extras import RealDictCursor


connection = psycopg2.connect(
    database = 'volebnikalkulacka',
    user = 'postgres',
    password = 'root',
    )

In [3]:
SQL_Query = pd.read_sql_query(
'''
SELECT zkratka,vysledek
FROM psp_data_hl_poslanec AS hp 
	INNER JOIN psp_data_poslanec AS p 
	ON hp.id_poslanec = p.id_poslanec 
    
    INNER JOIN psp_data_zarazeni AS z
    ON p.id_osoba = z.id_osoba
    
    INNER JOIN psp_data_organy as o
    ON z.id_of = o.id_organ
    
    INNER JOIN psp_data_osoby as os
    ON os.id_osoba = p.id_osoba
    
WHERE hp.id_hlasovani = 67306 --konkretni jedno hlasovani
AND z.cl_funkce = 0 --clenstvi
AND z.do_o IS NULL
AND o.organ_id_organ = 172 --Aktualni volebni obdobi
AND o.id_typ_organu = 1 --Klub


''', connection)

df = pd.DataFrame(SQL_Query)

In [4]:
df.groupby(['zkratka', 'vysledek']).tail()

,zkratka,vysledek
5,Piráti,@
9,Piráti,K
13,KDU-ČSL,@
17,ODS,@
20,Nezařaz,K
...,...,...
179,ODS,B
180,KSČM,A
181,Piráti,B
182,KDU-ČSL,@


In [5]:
df['zkratka']

0          ODS
1          ANO
2         KSČM
3          ANO
4          ANO
        ...   
179        ODS
180       KSČM
181     Piráti
182    KDU-ČSL
183        ANO
Name: zkratka, Length: 184, dtype: object

In [13]:
total_difference = 0
for party in df['zkratka'].unique():
    pro_df = df.query(f'zkratka=="{party}" & vysledek == "A"')
    proti_df = df.query(f'zkratka=="{party}" & (vysledek == "B" | vysledek == "N" )').count()
    members_count = len(df.query(f'zkratka=="{party}" & (vysledek == "A" | vysledek == "B" | vysledek == "N" ) ').index)
    
    if(members_count == 0):
        break
    
    pro_percentage = len(pro_df.index)/members_count
    proti_percentage = len(proti_df.index)/members_count
    party_difference = 1 - abs(pro_percentage-proti_percentage)
    
    total_difference += party_difference * (members_count/len(df.index)) * 100
    
total_difference

24.456521739130437

184